In [12]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.decomposition import PCA

### Mapping organ to protein

In [2]:
mapping_file_path = r"C:\Users\Romina\OneDrive\Desktop\Project\final_merged_protein_organ.csv"
mapping_df = pd.read_csv(mapping_file_path)


In [ ]:
main_data = pd.read_csv(r"C:\Users\Romina\OneDrive\Desktop\Project\Clean Notebooks and files\merged_proteomics_mri_all.csv")

C:\Users\Romina\AppData\Local\Temp\ipykernel_26200\2908534819.py:1: DtypeWarning: Columns (25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439

In [4]:
# Extract Protein Column Names from main_data
start_prot_idx = main_data.columns.get_loc("A1BG")
dataset_proteins = set(main_data.columns[start_prot_idx:])

In [5]:
# Find the Common Proteins Between Both Datasets
mapped_proteins = set(mapping_df['Protein'])
common_proteins = dataset_proteins.intersection(mapped_proteins)
print(f"Number of common proteins: {len(common_proteins)}")

Number of common proteins: 548


In [6]:
filtered_mapping_df = mapping_df[mapping_df['Protein'].isin(common_proteins)]


In [7]:
organ_to_proteins = filtered_mapping_df.groupby('Organ')['Protein'].apply(set).to_dict()

In [8]:
protein_count_per_organ = {organ: len(proteins) for organ, proteins in organ_to_proteins.items()}
protein_count_df = pd.DataFrame(list(protein_count_per_organ.items()), columns=["Organ", "Protein Count"])

In [9]:
print(f"Total organs mapped: {len(organ_to_proteins)}")
protein_count_df.head(15)

Total organs mapped: 15


,Organ,Protein Count
0,Adipose,3
1,Artery,9
2,Brain,147
3,Esophagus,3
4,Heart,7
5,Immune,95
6,Intestine,36
7,Kidney,8
8,Liver,80
9,Lung,10


In [ ]:
protein_to_organ_df = filtered_mapping_df.groupby("Protein")["Organ"].apply(lambda x: ", ".join(set(x))).reset_index()

# Save the mapping to a CSV file
protein_to_organ_df.to_csv("organ_protein_mapping.csv", index=False)
protein_to_organ_df

In [10]:
main_data

,eid,Sex,Ethnic_background,Age_at_recruitment,Diagnoses_main_ICD10,Diagnoses_main_ICD10_1,Diagnoses_main_ICD10_2,Diagnoses_main_ICD10_3,Diagnoses_main_ICD10_4,Diagnoses_main_ICD10_5,...,Volume of precuneus (RH),Volume of rostralanteriorcingulate (RH),Volume of rostralmiddlefrontal (RH),Volume of superiorfrontal (RH),Volume of superiorparietal (RH),Volume of superiortemporal (RH),Volume of supramarginal (RH),Volume of transversetemporal (RH),Volume of insula (RH),Imaging
0,1000024,0,1001.0,67,F019,G309,I48,I620,I639,M169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1000043,1,1001.0,65,NaN,NaN,NaN,NaN,NaN,NaN,...,11092.0,2790.0,14412.0,27460.0,11672.0,16217.0,11242.0,915.0,6353.0,1
2,1000156,0,1001.0,62,E871,H258,H269,R074,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1000217,1,1003.0,63,C060,I269,R509,R69,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1000309,1,4002.0,60,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52695,6023140,0,1001.0,55,H264,H269,H521,I259,I319,M201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
52696,6023206,1,2004.0,64,C447,D509,E831,G562,I839,K219,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
52697,6023457,1,1001.0,48,D125,I841,I848,K621,K640,Q433,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
52698,6023548,1,1001.0,62,C155,C159,C160,C675,C679,K918,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


## **Lasso Model: Organ-Specific, Organismal, and All-Protein Models**

## the Organ Age Gap?
- Difference between **predicted organ age** and **actual age**.
- **Positive values** → Organ is aging **faster** than expected.
- **Negative values** → Organ is aging **slower** than expected.

## Data Preprocessing & Training
- **Proteomics data** used to predict age.
- **Train/Test Split** based on Imaging column.
- **K-Fold Cross-Validation (5 folds)** to:
  - Avoid overfitting.
  - Find optimal model parameters.

## Machine Learning
- **LASSO Regression**:
  - Selects important proteins.
  - Shrinks non-relevant features to zero.
- **Steps**:
  1. Feature selection (organ-specific proteins).
  2. Data normalization & missing value imputation.
  3. Model training (LASSO with Cross-Validation).
  4. Prediction of age → Calculation of **organ age gap**.

## Models Trained
 **Organ-Specific Models** (e.g., brain, heart, liver).  
 **Organismal Model** (all mapped proteins).  
 **All-Protein Model** (entire proteomics dataset).

## Model Performance
- **R² Score**: How well the model predicts age.
- **Mean Age Gap**: Are organs aging faster/slower?
- **Standard deviation** of predictions.

In [ ]:
import gc  # Garbage collection
print("Preparing data...")

train_data = main_data[main_data["Imaging"] == 0].copy()
test_data = main_data[main_data["Imaging"] == 1].copy()

y_train_full = train_data["Age_at_recruitment"].copy()
y_test_full = test_data["Age_at_recruitment"].copy()

# One-Hot Encode "Sex" to ensure numerical values
encoder = OneHotEncoder(drop="first", sparse_output=False)
train_sex_encoded = encoder.fit_transform(train_data[["Sex"]])
test_sex_encoded = encoder.transform(test_data[["Sex"]])

train_sex_df = pd.DataFrame(train_sex_encoded, index=train_data.index, columns=encoder.get_feature_names_out(["Sex"]))
test_sex_df = pd.DataFrame(test_sex_encoded, index=test_data.index, columns=encoder.get_feature_names_out(["Sex"]))

# Drop original "Sex" column and add encoded values
train_data = train_data.drop(columns=["Sex"]).join(train_sex_df)
test_data = test_data.drop(columns=["Sex"]).join(test_sex_df)

organ_age_gaps_all = pd.DataFrame(index=main_data.index)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

model_results = {}

# Organ-Specific Models
print("Training Organ-Specific Models...")
organ_results = {}

for organ, proteins in tqdm(organ_to_proteins.items(), desc="Processing Organs"):
    valid_proteins = [protein for protein in proteins if protein in main_data.columns]
    if not valid_proteins:
        print(f"No valid proteins found for: {organ}")
        continue

    X_train = train_data[valid_proteins].copy().join(train_sex_df)
    X_test = test_data[valid_proteins].copy().join(test_sex_df)

    organ_age_gap = pd.Series(0.0, index=main_data.index)  

    for train_index, test_index in kf.split(X_train):
        X_train_cv, X_test_cv = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_cv, y_test_cv = y_train_full.iloc[train_index], y_train_full.iloc[test_index]

        imputer = SimpleImputer(strategy="mean")
        scaler = StandardScaler()

        X_train_scaled = scaler.fit_transform(imputer.fit_transform(X_train_cv))
        X_test_scaled = scaler.transform(imputer.transform(X_test_cv))

        lasso = LassoCV(cv=5, random_state=42, n_alphas=50)
        lasso.fit(X_train_scaled, y_train_cv)

        y_test_pred = lasso.predict(X_test_scaled)
        organ_age_gap.loc[X_test_cv.index] = y_test_pred - y_test_cv 

    organ_age_gaps_all[organ] = organ_age_gap

    # Final Model Training & Out-of-Sample Predictions for MRI Participants
    X_train_final_scaled = scaler.fit_transform(imputer.fit_transform(X_train))
    X_test_final_scaled = scaler.transform(imputer.transform(X_test))

    lasso.fit(X_train_final_scaled, y_train_full)
    y_test_pred_final = lasso.predict(X_test_final_scaled)
    organ_age_gaps_all.loc[test_data.index, organ] = y_test_pred_final - y_test_full

    organ_results[organ] = {
        "Optimal Alpha": lasso.alpha_,
        "Train R²": lasso.score(X_train_final_scaled, y_train_full),
        "Test R²": lasso.score(X_test_final_scaled, y_test_full),
        "Test Mean Age Gap": (y_test_pred_final - y_test_full).mean(),
        "Test Std Dev Age Gap": (y_test_pred_final - y_test_full).std(),
    }

    gc.collect()

model_results.update(organ_results)

# Organismal Model
print("Training the Organismal Model...")

all_mapped_proteins = set().union(*organ_to_proteins.values())
valid_organism_wide_proteins = [protein for protein in all_mapped_proteins if protein in main_data.columns]

X_organism_train = train_data[valid_organism_wide_proteins].copy().join(train_sex_df)
X_organism_test = test_data[valid_organism_wide_proteins].copy().join(test_sex_df)

organism_age_gap = pd.Series(0.0, index=main_data.index)

for train_index, test_index in kf.split(X_organism_train):
    X_train_cv, X_test_cv = X_organism_train.iloc[train_index], X_organism_train.iloc[test_index]
    y_train_cv, y_test_cv = y_train_full.iloc[train_index], y_train_full.iloc[test_index]

    X_train_scaled = scaler.fit_transform(imputer.fit_transform(X_train_cv))
    X_test_scaled = scaler.transform(imputer.transform(X_test_cv))

    lasso.fit(X_train_scaled, y_train_cv)
    y_test_pred = lasso.predict(X_test_scaled)
    organism_age_gap.loc[X_test_cv.index] = y_test_pred - y_test_cv

organ_age_gaps_all["Organism_Model"] = organism_age_gap

# All-Protein Model
print("Training the All-Protein Model...")

# Select all columns between "A1BG" and "ZPR1"
protein_start = main_data.columns.get_loc("A1BG")  
protein_end = main_data.columns.get_loc("ZPR1") + 1  # Include ZPR1
protein_columns = main_data.columns[protein_start:protein_end].tolist()

# Ensure "Sex" exists in train_data and test_data
if "Sex" not in train_data.columns:
    print(" Warning: 'Sex' column not found in train_data! Adding it manually.")
    train_data["Sex"] = main_data["Sex"]
    
if "Sex" not in test_data.columns:
    print(" Warning: 'Sex' column not found in test_data! Adding it manually.")
    test_data["Sex"] = main_data["Sex"]


X_all_train = train_data[protein_columns + ["Sex"]].copy()
X_all_test = test_data[protein_columns + ["Sex"]].copy()

# Initialize storage for age gap predictions
all_protein_age_gap = pd.Series(0.0, index=main_data.index)

# Cross-validation loop
for train_index, test_index in kf.split(X_all_train):
    X_train_cv, X_test_cv = X_all_train.iloc[train_index], X_all_train.iloc[test_index]
    y_train_cv, y_test_cv = y_train_full.iloc[train_index], y_train_full.iloc[test_index]

    X_train_scaled = scaler.fit_transform(imputer.fit_transform(X_train_cv))
    X_test_scaled = scaler.transform(imputer.transform(X_test_cv))

    lasso.fit(X_train_scaled, y_train_cv)
    y_test_pred = lasso.predict(X_test_scaled)
    all_protein_age_gap.loc[X_test_cv.index] = y_test_pred - y_test_cv

# Store results
organ_age_gaps_all["All_Protein_Model"] = all_protein_age_gap

comparison_results = pd.DataFrame.from_dict(model_results, orient="index")
comparison_results.to_csv("organ_model_performance.csv", index=True)
comparison_results

organ_age_gaps_all["eid"] = main_data["eid"]
organ_age_gaps_all.to_csv("organ_age_gaps_final.csv", index=False)



Preparing data...
Training Organ-Specific Models...


Processing Organs: 100%|██████████| 15/15 [01:46<00:00,  7.08s/it]


Training the Organismal Model...
Training the All-Protein Model...
🚨 Warning: 'Sex' column not found in train_data! Adding it manually.
🚨 Warning: 'Sex' column not found in test_data! Adding it manually.


c:\Users\Romina\OneDrive\Documents\learn_python\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 536.0821960241301, tolerance: 206.67857460611978
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\Romina\OneDrive\Documents\learn_python\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 698.7295636988129, tolerance: 206.67857460611978
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\Romina\OneDrive\Documents\learn_python\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1001.508524820907, tolerance: 206.67857460611978
  model = cd_fast.enet_coordinate_descent_gram(
c:\User

In [13]:
pc1_data = main_data.copy()

for organ, proteins in tqdm(organ_to_proteins.items(), desc="Processing Organs"):
    
    proteins = list(proteins) 
    

    valid_proteins = [protein for protein in proteins if protein in main_data.columns]
    
    if not valid_proteins:
        print(f"No valid proteins found for: {organ}")
        continue  


    organ_data = main_data[valid_proteins].copy()

    scaler = StandardScaler()
    organ_data_scaled = scaler.fit_transform(organ_data.fillna(0))  # Fill NaN with 0 before PCA

    # Perform PCA and extract PC1
    pca = PCA(n_components=1)
    pc1_values = pca.fit_transform(organ_data_scaled)

    # Store PC1 in main_data
    pc1_data[f"{organ}_PC1"] = pc1_values.flatten()

pc1_data.to_csv("main_data_with_PC1.csv", index=False)


Processing Organs: 100%|██████████| 15/15 [00:01<00:00,  9.34it/s]


In [15]:
pc1_data

,eid,Sex,Ethnic_background,Age_at_recruitment,Diagnoses_main_ICD10,Diagnoses_main_ICD10_1,Diagnoses_main_ICD10_2,Diagnoses_main_ICD10_3,Diagnoses_main_ICD10_4,Diagnoses_main_ICD10_5,...,Immune_PC1,Intestine_PC1,Kidney_PC1,Liver_PC1,Lung_PC1,Muscle_PC1,Pancreas_PC1,Skin_PC1,Stomach_PC1,Whole Blood_PC1
0,1000024,0,1001.0,67,F019,G309,I48,I620,I639,M169,...,1.303253,2.681153,-0.433052,-0.652987,0.936314,-1.402988,5.988106,0.625040,0.645664,-2.800662
1,1000043,1,1001.0,65,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.093359,-1.256863,-0.113210,-1.756817,-0.144261,-0.509540,-1.177616,0.255515,2.993833,4.432281
2,1000156,0,1001.0,62,E871,H258,H269,R074,NaN,NaN,...,-4.361638,-3.044292,-0.332554,3.115975,1.625565,-0.375860,-0.968636,-1.046977,-1.138501,-3.673011
3,1000217,1,1003.0,63,C060,I269,R509,R69,NaN,NaN,...,-0.447104,7.067998,1.563478,2.234244,0.274612,-0.747867,6.070332,0.846050,2.772288,1.098881
4,1000309,1,4002.0,60,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.397951,1.143069,1.601042,-0.642389,-2.131549,0.121562,3.338186,0.441217,0.139011,0.523095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52695,6023140,0,1001.0,55,H264,H269,H521,I259,I319,M201,...,1.380405,-0.510587,-0.439088,3.353094,0.075997,-4.492066,-2.802748,-1.513215,-3.233504,1.171153
52696,6023206,1,2004.0,64,C447,D509,E831,G562,I839,K219,...,2.084492,1.225070,0.708167,2.324371,-1.656841,6.618258,3.113782,2.449672,-0.214516,2.239805
52697,6023457,1,1001.0,48,D125,I841,I848,K621,K640,Q433,...,3.311513,-1.456235,0.433816,-2.579610,-0.591359,0.930197,-4.764542,1.233305,-2.028132,3.438137
52698,6023548,1,1001.0,62,C155,C159,C160,C675,C679,K918,...,-1.260204,6.707043,0.612205,4.719469,0.843217,1.696371,0.226650,2.206144,5.011710,-4.524493
